In [1]:
import numpy as np
from scipy.optimize import fsolve
import casadi as ca
import plotly.graph_objects as go
import optuna
from plotly.subplots import make_subplots
import time
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from scipy.interpolate import griddata
from libs.simulationn import Simulation
from libs.Model import MyModel
from libs.Interpolation import Interpolation

/home/matheus/.local/lib/python3.12/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [ ]:
np.random.seed(42)
print(np.random.seed)

# Constantes
A1 = (2.6)*(10**-3)
Lc = 2
kv = 0.38
P1 = 4.5
P_out = 5
C = 479

timestep = 3 # Passos no passado para prever o próximo
nAlphas = 40 # Número de vezes que o Alfa irá mudar, considere o treino e os testes.
alphas = np.random.uniform(0.35,0.65, nAlphas+1) # Abertura da válvula
N_RotS = np.random.uniform(27e3, 5e4, nAlphas+1)
epochs = 1000
nData = 1000
nAlphasTeste = 10
nDataTeste = 400
perturb = 1e-4
tempo = 60
tempoTeste = 40
dt = 0.1 # Tempo amostral

# Variáveis auxiliares
interval = [np.linspace(i * tempo, (i + 1) * tempo, nData) for i in range(nAlphas)]
interval_test = [np.linspace(i * tempoTeste, (i + 1) * tempoTeste, nDataTeste) for i in range(nAlphasTeste)]
massFlowrate = []
PlenumPressure = []
alpha_values = []
RNN_train = []
RNN_trainFut = []

In [ ]:
massFlowrateTeste = []
PlenumPressureTeste = []
RNN_test = []
x_test = []
alpha_valuesTeste = []
aux1 = []
aux2 = []
alphasTeste = np.random.uniform(0.35,0.65, nAlphasTeste) # Abertura da válvula
N_RotSTeste = np.random.uniform(27e3, 5e4, nAlphasTeste+1)

sim = Simulation(A1, Lc, kv, P1, P_out, C, alphasTeste,N_RotSTeste, nAlphasTeste, nDataTeste, perturb, tempoTeste, dt, interpolation, timestep)

sim.run()

massFlowrateTeste = sim.massFlowrate
PlenumPressureTeste = sim.PlenumPressure
alpha_valuesTeste = sim.alpha_values
x_test = sim.X_train
print(sim.time)


massFlowrateTeste = np.array(massFlowrateTeste)
PlenumPressureTeste = np.array(PlenumPressureTeste)
interval3 = np.linspace(0, tempoTeste*nAlphasTeste, len(x_test))